# Requirement

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rounakbanik/the-movies-dataset




mkdir: cannot create directory ‘/root/.kaggle’: File exists
the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:

import zipfile

In [3]:
zip_ref = zipfile.ZipFile("the-movies-dataset.zip", "r")
zip_ref.extractall()
zip_ref.close()

# First Look

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("/content/movies_metadata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# Cleaning 

In [9]:
data = data[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

In [10]:
data.shape

(45466, 6)

## Extracting Year

In [11]:
#Helper function to convert NaT to 0 and all other years to integers.
def ConvertToInt(x):
    try:
        return int(x)
    except:
        return 0

In [12]:
def ExtractDate(data):
  #Convert release_date into pandas datetime format
  data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')

  #Extract year from the datetime
  data['year'] = data['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

  data['year'] = data['year'].apply(ConvertToInt)

  data = data.drop('release_date', axis=1)

  return data

## Extracting Genres

In [13]:
from ast import literal_eval

In [14]:
def ExtractGenres(data):
  #Convert all NaN into stringified empty lists
  data['genres'] = data['genres'].fillna('[]')

  #Apply literal_eval to convert stringified empty lists to the list object
  data['genres'] = data['genres'].apply(literal_eval)

  #Convert list of dictionaries to a list of strings
  data['genres'] = data['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])
  
  return data

## Exploding Genres

In [15]:
def ExplodingGenres(data):
    #Create a new feature by exploding genres
  newFeature = data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

  #Name the new feature as 'genre'
  newFeature.name = 'genre'

  #Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
  data = data.drop('genres', axis=1).join(newFeature)

  return data

In [16]:
def ScoreCalculating(data,percentile=0.8):
     #Compute the values of C and m for the filtered movies
    C = data['vote_average'].mean()
    m = data['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = data.copy().loc[data['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

## Pipeline

In [17]:
def Pipeline(data):
  data =  ExtractDate(data)
  data =  ExtractGenres(data)
  #data = ExplodingGenres(data)
  #data = ScoreCalculating(data,0.8)
  return data

In [18]:
data = Pipeline(data)

In [19]:
data.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


In [20]:
orig_df = pd.read_csv('/content/movies_metadata.csv', low_memory=False)
#Add the useful features into the cleaned dataframe
data['overview'], data['id'] = orig_df['overview'], orig_df['id']

In [21]:
data.shape

(45466, 8)

# TF-IDF

In [22]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english',max_features=45466)

#Replace NaN with an empty string
data['overview'] = data['overview'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(data['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 45466)

In [ ]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=data, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data['title'].iloc[movie_indices]
#Get recommendations for The Lion King
content_recommender('The Lion King')